In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Peptide Dataset Analysis\n",
    "\n",
    "This notebook analyzes the peptide dataset for StructDiff training."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from pathlib import Path\n",
    "import json\n",
    "\n",
    "# Setup\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (10, 6)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Load Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load processed data\n",
    "data_dir = Path('../data/processed')\n",
    "\n",
    "train_df = pd.read_csv(data_dir / 'train.csv')\n",
    "val_df = pd.read_csv(data_dir / 'val.csv')\n",
    "test_df = pd.read_csv(data_dir / 'test.csv')\n",
    "\n",
    "# Load statistics\n",
    "with open(data_dir / 'statistics.json', 'r') as f:\n",
    "    stats = json.load(f)\n",
    "\n",
    "print(f\"Train samples: {len(train_df)}\")\n",
    "print(f\"Val samples: {len(val_df)}\")\n",
    "print(f\"Test samples: {len(test_df)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Length Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot length distribution\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "# Histogram\n",
    "all_lengths = pd.concat([train_df['length'], val_df['length'], test_df['length']])\n",
    "ax1.hist(all_lengths, bins=30, alpha=0.7, edgecolor='black')\n",
    "ax1.set_xlabel('Sequence Length')\n",
    "ax1.set_ylabel('Count')\n",
    "ax1.set_title('Peptide Length Distribution')\n",
    "\n",
    "# Box plot by dataset\n",
    "data_lengths = [\n",
    "    train_df['length'].values,\n",
    "    val_df['length'].values,\n",
    "    test_df['length'].values\n",
    "]\n",
    "ax2.boxplot(data_lengths, labels=['Train', 'Val', 'Test'])\n",
    "ax2.set_ylabel('Sequence Length')\n",
    "ax2.set_title('Length Distribution by Split')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Peptide Type Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze peptide types\n",
    "if 'peptide_type' in train_df.columns:\n",
    "    # Count types\n",
    "    type_counts = train_df['peptide_type'].value_counts()\n",
    "    \n",
    "    # Plot\n",
    "    plt.figure(figsize=(8, 6))\n",
    "    type_counts.plot(kind='bar')\n",
    "    plt.xlabel('Peptide Type')\n",
    "    plt.ylabel('Count')\n",
    "    plt.title('Distribution of Peptide Types (Training Set)')\n",
    "    plt.xticks(rotation=45)\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"No peptide type information available\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Amino Acid Composition"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze amino acid frequencies\n",
    "aa_freq = stats['total']['amino_acid_frequencies']\n",
    "\n",
    "# Natural frequencies for comparison\n",
    "natural_freq = {\n",
    "    'A': 0.074, 'C': 0.025, 'D': 0.054, 'E': 0.054,\n",
    "    'F': 0.047, 'G': 0.074, 'H': 0.026, 'I': 0.068,\n",
    "    'K': 0.058, 'L': 0.099, 'M': 0.025, 'N': 0.045,\n",
    "    'P': 0.039, 'Q': 0.034, 'R': 0.052, 'S': 0.057,\n",
    "    'T': 0.051, 'V': 0.073, 'W': 0.013, 'Y': 0.032\n",
    "}\n",
    "\n",
    "# Create comparison plot\n",
    "amino_acids = sorted(set(aa_freq.keys()) & set(natural_freq.keys()))\n",
    "dataset_freqs = [aa_freq.get(aa, 0) for aa in amino_acids]\n",
    "natural_freqs = [natural_freq.get(aa, 0) for aa in amino_acids]\n",
    "\n",
    "x = np.arange(len(amino_acids))\n",
    "width = 0.35\n",
    "\n",
    "fig, ax = plt.subplots(figsize=(12, 6))\n",
    "ax.bar(x - width/2, dataset_freqs, width, label='Dataset', alpha=0.8)\n",
    "ax.bar(x + width/2, natural_freqs, width, label='Natural', alpha=0.8)\n",
    "\n",
    "ax.set_xlabel('Amino Acid')\n",
    "ax.set_ylabel('Frequency')\n",
    "ax.set_title('Amino Acid Frequency Comparison')\n",
    "ax.set_xticks(x)\n",
    "ax.set_xticklabels(amino_acids)\n",
    "ax.legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Calculate KL divergence\n",
    "from scipy.stats import entropy\n",
    "kl_div = entropy(dataset_freqs, natural_freqs)\n",
    "print(f\"KL Divergence from natural distribution: {kl_div:.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Structure Availability"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check structure availability\n",
    "if 'has_structure' in train_df.columns:\n",
    "    structure_stats = {\n",
    "        'Train': train_df['has_structure'].mean(),\n",
    "        'Val': val_df['has_structure'].mean(),\n",
    "        'Test': test_df['has_structure'].mean()\n",
    "    }\n",
    "    \n",
    "    # Plot\n",
    "    plt.figure(figsize=(8, 6))\n",
    "    plt.bar(structure_stats.keys(), structure_stats.values())\n",
    "    plt.ylabel('Fraction with Structure')\n",
    "    plt.title('Structure Availability by Split')\n",
    "    plt.ylim(0, 1)\n",
    "    \n",
    "    # Add percentage labels\n",
    "    for i, (k, v) in enumerate(structure_stats.items()):\n",
    "        plt.text(i, v + 0.02, f'{v*100:.1f}%', ha='center')\n",
    "    \n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"No structure information available\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Sequence Complexity Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def calculate_complexity(sequence):\n",
    "    \"\"\"Calculate sequence complexity (Shannon entropy)\"\"\"\n",
    "    if len(sequence) == 0:\n",
    "        return 0\n",
    "    \n",
    "    # Count amino acids\n",
    "    counts = {}\n",
    "    for aa in sequence:\n",
    "        counts[aa] = counts.get(aa, 0) + 1\n",
    "    \n",
    "    # Calculate entropy\n",
    "    entropy = 0\n",
    "    for count in counts.values():\n",
    "        p = count / len(sequence)\n",
    "        if p > 0:\n",
    "            entropy -= p * np.log2(p)\n",
    "    \n",
    "    return entropy\n",
    "\n",
    "# Calculate complexity for all sequences\n",
    "train_df['complexity'] = train_df['sequence'].apply(calculate_complexity)\n",
    "\n",
    "# Plot complexity distribution\n",
    "plt.figure(figsize=(10, 6))\n",
    "plt.hist(train_df['complexity'], bins=30, alpha=0.7, edgecolor='black')\n",
    "plt.xlabel('Sequence Complexity (Shannon Entropy)')\n",
    "plt.ylabel('Count')\n",
    "plt.title('Distribution of Sequence Complexity')\n",
    "plt.axvline(train_df['complexity'].mean(), color='red', linestyle='--', \n",
    "           label=f'Mean: {train_df[\"complexity\"].mean():.2f}')\n",
    "plt.legend()\n",
    "plt.show()\n",
    "\n",
    "# Complexity vs length\n",
    "plt.figure(figsize=(10, 6))\n",
    "plt.scatter(train_df['length'], train_df['complexity'], alpha=0.5)\n",
    "plt.xlabel('Sequence Length')\n",
    "plt.ylabel('Sequence Complexity')\n",
    "plt.title('Sequence Complexity vs Length')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Save Analysis Summary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create analysis summary\n",
    "summary = {\n",
    "    'dataset_sizes': {\n",
    "        'train': len(train_df),\n",
    "        'val': len(val_df),\n",
    "        'test': len(test_df),\n",
    "        'total': len(train_df) + len(val_df) + len(test_df)\n",
    "    },\n",
    "    'length_statistics': stats['total']['length_stats'],\n",
    "    'amino_acid_kl_divergence': float(kl_div),\n",
    "    'complexity_stats': {\n",
    "        'mean': float(train_df['complexity'].mean()),\n",
    "        'std': float(train_df['complexity'].std()),\n",
    "        'min': float(train_df['complexity'].min()),\n",
    "        'max': float(train_df['complexity'].max())\n",
    "    }\n",
    "}\n",
    "\n",
    "# Save summary\n",
    "with open('dataset_analysis_summary.json', 'w') as f:\n",
    "    json.dump(summary, f, indent=2)\n",
    "\n",
    "print(\"Analysis complete!\")\n",
    "print(json.dumps(summary, indent=2))"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}